In [2]:
import requests
import urllib
from bs4 import BeautifulSoup
import time
from urllib.robotparser import RobotFileParser
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import re

headers = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
    else: #bot 접근 못하게 막은데있어어 else뺌
        try:
            resp = requests.request(method, url,
                   params=params if method=='GET' else {},
                   data=params if method=='POST' else {},
                   headers=headers)
            resp.raise_for_status()
        except HTTPError as e:
            if limit > 0 and e.response.status_code >= 500: # 500 에러 재귀 ;너무많은 트래픽 안생기도록
                print(limit)
                time.sleep(300) # => random, 1초에 1번
                resp = download(url, params, headers, method, limit-1)
            else:
                print('[{}] '.format(e.response.status_code) + url)
                print(e.response.status_code)
                print(e.response.reason)
                print(e.response.headers)
    return resp

## 다운로드 함수 바꾸기

- robots.txt로 접근 가능 여부 판단
- error 뜨면 못 가져오니깐 딴거로 고고

In [12]:
download('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24',headers=headers)

<Response [200]>

# 메인화면꺼만 가져와본거

In [8]:
from selenium import webdriver
driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24')

page = 1
url = 'http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24&page='
driver.get(url+str(page))
# driver.execute_script('console.log("hi")')
leng = len(driver.find_elements_by_class_name('other'))

for r in range(leng+1):    
    for i, _ in enumerate(driver.find_elements_by_css_selector('img')):
        if _.get_attribute('src').split('/')[4]=='product':
            urllib.request.urlretrieve(_.get_attribute('src'),"./img/" + str(r) + 'bakery' +str(i)+'.jpg')
    if r == leng:
        break
    page+=1
    driver.get(url+str(page))
    time.sleep(2)

# 상품 세부사진 다 가져오기

In [9]:
from selenium import webdriver
# driver = webdriver.Chrome('/Users/Gyu/Desktop/mywork/KData/chromedriver')
driver.get('http://xn--hg3bqa693eb6a.com/product/list.html?cate_no=24')
total= int(driver.find_element_by_css_selector('p[class=prd_count] > strong').text.strip())
total
# total : 메인 화면에 30개라고 나와있음

30

In [11]:
url = 'http://xn--hg3bqa693eb6a.com/product/%EC%95%BC%EB%AF%B8%EC%9A%94%EB%B0%80-8%E7%84%A1-8%EA%B3%A1-%EC%83%9D%EC%8C%80%EC%8B%9D%EB%B9%B5-%EC%8C%80%EB%B9%B5%EB%B9%84%EA%B1%B4%EB%B9%B5%EB%B9%84%EA%B1%B4%EC%8C%80%EB%B9%B5%EA%B1%B4%EA%B0%95%EB%B9%B5%EB%8B%A4%EC%9D%B4%EC%96%B4%ED%8A%B8%EB%B9%B5%EB%B9%84%EA%B1%B4/69/category/24/display/1/'
driver.get(url)

for i in range(total):
    for j, _ in enumerate(driver.find_elements_by_css_selector('img[class=ThumbImage]')):
        urllib.request.urlretrieve(_.get_attribute('src'),"./img/"+ 'bread_' +str(i)+ '_' + str(j) + '.jpg')
    driver.find_element_by_css_selector('span[class=next] > a').click()
    time.sleep(3)  
    # 이거 어느 정도 시간을 둬야 사진을 제대로 가져옴

In [14]:
driver.close()

# pixel 웹사이트에서 사진 가져오기

- DB 만들기

In [1]:
import sqlite3
con = sqlite3.connect('food_image.db')
cur = con.cursor()

cur.executescript('''
    DROP TABLE IF EXISTS food_image;
    CREATE TABLE food_image(
        FOOD_NAME    TEXT,
        FOOD_IMAGE    TEXT
    );
''')

In [4]:
import requests
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

In [10]:
from bs4 import BeautifulSoup
page = requests.get('https://pixabay.com/images/search/vegan/')
soup = BeautifulSoup(page.text, 'html.parser')

<Response [403]>

In [11]:
page.status_code

403

In [13]:
# print(soup.prettify())
soup.find_all(id='item')

[]

In [ ]:
images = soup.findAll('img')
for image in images:
    #print image source
    print image['src']
    #print alternate text
    print image['alt']

In [ ]:
re.findall(r'([$]\d+[.]\d+)', resp.text)

In [20]:
import urllib
from selenium import webdriver
driver = webdriver.Chrome('/Users/sohyun/Downloads/chromedriver')
driver.get('https://pixabay.com/images/search/vegan/')
  
for i, _ in enumerate(driver.find_elements_by_css_selector('img')):
    print(_.get_attribute('src'))


https://cdn.pixabay.com/photo/2017/05/23/22/36/vegetables-2338824__340.jpg
https://cdn.pixabay.com/photo/2017/09/16/19/21/salad-2756467__340.jpg
https://cdn.pixabay.com/photo/2015/10/02/15/59/olive-oil-968657__340.jpg
https://cdn.pixabay.com/photo/2015/12/09/17/11/vegetables-1085063__340.jpg
https://cdn.pixabay.com/photo/2018/06/23/16/22/romanesco-3493007__340.jpg
https://cdn.pixabay.com/photo/2016/11/06/23/31/breakfast-1804457__340.jpg
https://cdn.pixabay.com/photo/2014/12/21/23/39/coconuts-575780__340.png
https://cdn.pixabay.com/photo/2017/11/12/19/22/water-2943518__340.jpg
https://cdn.pixabay.com/photo/2016/09/12/08/22/breakfast-1663295__340.jpg
https://cdn.pixabay.com/photo/2016/10/25/13/16/pumpkin-1768857__340.jpg
https://cdn.pixabay.com/photo/2018/09/23/09/31/smoothie-3697014__340.jpg
https://cdn.pixabay.com/photo/2017/01/25/18/08/vegetables-2008578__340.png
https://cdn.pixabay.com/photo/2017/12/29/16/34/fruit-3048001__340.jpg
https://cdn.pixabay.com/photo/2017/07/27/17/30/tray-2

In [21]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=84.0.4147.105)
